In [1]:
from wordcloud import WordCloud
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib 
from scipy import optimize
from scipy import stats
plt.style.use("ggplot")
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
#plt.rcParams['font.family'] = 'IPAexGothic'

import warnings
warnings.filterwarnings('ignore') # warningが出ないように設定
pd.set_option("display.max_rows", None) # pandasの表示上限をなくす
pd.set_option("display.max_columns", None) # pandasの表示上限をなくす

import gensim

model = gensim.models.Word2Vec.load("latest-ja-word2vec-gensim-model/word2vec.gensim.model")

In [2]:
import pickle

f = open("9_relatoinvec_list",'rb')
mean_vec_list = pickle.load(f)

In [3]:
len(mean_vec_list)

1275

In [4]:
# 入力単語

imput_word_list = ['太陽光発電', "住宅"]

In [5]:
# 関係ベクトルに入力単語のベクトルを加算, 減算

result_vec_list  = []

for w in imput_word_list:

    w_to_v = model.wv[w]

    for mean_vec in mean_vec_list:
        result_vec_plus = w_to_v - mean_vec 
        result_vec_minus =   w_to_v + mean_vec 
    
        result_vec_list.append(result_vec_plus)
        result_vec_list.append(result_vec_minus)

In [6]:
# 推定ベクトル数

len(result_vec_list)

5100

In [8]:
# 登録済固有名詞リスト

f = open("./koyuu_noun_list.binaryfile",'rb')
noun_list = pickle.load(f)

In [9]:
# 推定ベクトルと類似度が高い単語をn個上位表示

import numpy as np

output_word_list = []
max_n = 10

for vec in result_vec_list:
    most_similar = np.array(list(model.wv.most_similar([vec], [], max_n)))
    
    output_word_list.extend(most_similar[:,0].tolist())

In [10]:
#インプットした単語と同じ単語は除去

for imput_word in imput_word_list:
    output_word_list = [output_word for output_word in output_word_list if output_word != imput_word]
    print(len(output_word_list))

48550
46100


In [11]:
# 固有名詞リスト外の単語は除去
# これをすると単語が消える=>普通名詞の関係ベクトルは普通名詞を推定しやすい

output_word_list = [output_word for output_word in output_word_list if noun_list.count(output_word)>=1]

In [12]:
#共起単語と共起回数をdic化してまとめる

import collections

c = collections.Counter(output_word_list)
c = sorted(c.items(), key=lambda x:x[1])
c

[('リアビュー', 1),
 ('温水', 2),
 ('リアプロジェクションテレビ', 2),
 ('原水', 2),
 ('長屋', 2),
 ('関西電力', 4),
 ('田畑', 11),
 ('町屋', 16),
 ('もんじゅ', 18),
 ('コジェネレーション', 21),
 ('センサ', 31),
 ('エコキュート', 37),
 ('サッフォー', 200),
 ('澹', 200),
 ('ヒートポンプ', 639)]

In [13]:
for taple in c:
    word = taple[0]
    num = taple[1]